In [384]:
import requests, json
from bs4 import BeautifulSoup   
import numpy as np 
import pandas as pd
import time
import datetime

In [385]:
RANDOM_SEED = 42

In [386]:
test_cars=pd.read_csv('test_cars.csv')
test_cars.head(3)

/Users/do/bin/homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,12,16,17,19,20,32,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,Руль,Состояние,Таможня,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [387]:
test_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              34686 non-null  object 
 1   brand                 34686 non-null  object 
 2   car_url               34686 non-null  object 
 3   color                 34686 non-null  object 
 4   complectation_dict    6418 non-null   object 
 5   description           34686 non-null  object 
 6   engineDisplacement    34686 non-null  object 
 7   enginePower           34686 non-null  object 
 8   equipment_dict        24694 non-null  object 
 9   fuelType              34683 non-null  object 
 10  image                 34686 non-null  object 
 11  mileage               34686 non-null  object 
 12  modelDate             34686 non-null  object 
 13  model_info            34686 non-null  object 
 14  model_name            34686 non-null  object 
 15  name               

In [388]:
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info',
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
test_cars.head(3)

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,productionDate,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,бензин,74000,2013,OCTAVIA,...,2014,LIFTBACK ROBOT 1.2,роботизированная,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,бензин,60563,2017,OCTAVIA,...,2017,LIFTBACK MECHANICAL 1.6,механическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,бензин,88000,2013,SUPERB,...,2014,LIFTBACK ROBOT 1.8,роботизированная,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [389]:
test_cars[['brand', 'model_name']].groupby("brand")["model_name"].unique().head

<bound method NDFrame.head of brand
AUDI          [Q5, R8, Q7, Q3, A4, A3, A5, A6, RS7, S6, RS6,...
BMW           [X5, X6, X7, X3, 5ER, X2, Z4, X1, 6ER, 7ER, X5...
HONDA         [CR_V, ACCORD, PILOT, CIVIC, ODYSSEY, STREAM, ...
INFINITI      [JX, FX, G35, Q50, QX56, QX50, QX80, EX, Q70, ...
LEXUS         [LS, GS, ES, IS, LX, RX, NX, GX, UX, CT, LM, R...
MERCEDES      [V_KLASSE, C_KLASSE, GLE_KLASSE, E_KLASSE, S_K...
MITSUBISHI    [PAJERO_SPORT, OUTLANDER, L200, MONTERO, LANCE...
NISSAN        [PATROL, TEANA, QASHQAI, AVENIR, X_TRAIL, MURA...
SKODA         [OCTAVIA, SUPERB, OCTAVIA_RS, YETI, KODIAQ, RA...
TOYOTA        [CAMRY, RAV_4, AVENSIS, GT86, LAND_CRUISER, CO...
VOLKSWAGEN    [TOUAREG, TIGUAN, PASSAT, AMAROK, POLO, CADDY,...
VOLVO         [V60_CROSS_COUNTRY, XC40, XC60, C30, V50, XC90...
Name: model_name, dtype: object>

In [390]:
url='https://auto.ru/-/ajax/desktop/listing/'

In [391]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    
    'content-type': 'application/json',
    'Cookie': '_csrf_token=d8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010; autoru_sid=a%3Ag5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f%7C1607183995440.604800.lS7lenbdJu7TVJgQ9a2fBw.M4uhYka1yjkPXI7TXYAIHoc1bAMl16bHMa5Rel1JyiM; autoruuid=g5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f; autoru_gdpr=1; suid=bc138c1d5a6554d5f028f6fec11a91da.a9ca18211fb27ebefea2fe04a11546b6; from=direct; yuidcs=1; crookie=aAs/ROiBsnuk/909OZNBhxIcXPqhyXpNRRJ0N6Sk1kI/Eaknz7W1f3rW2MUp05I5lp/zbVBPKBkNgTKZqk8ULF8Q0Vg=; cmtchd=MTYwNzI1MDk3ODU4MQ==; popup_new_user=new; proven_owner_popup=1; bltsr=1; yuidlt=1; yandexuid=1370723801540629363; X-Vertis-DC=vla; listing_view_session={}; listing_view=%7B%22output_type%22%3A%22models_list%22%2C%22version%22%3A1%7D; new-controls-notify-hidden=true; from_lifetime=1607289350518',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202012.04.170215',
    'x-client-date': '1607289359144',
    'x-csrf-token': 'd8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010',
    'x-page-request-id': '52d78c631a6cdbf362cebe6be367d327',
    'x-requested-with': 'fetch'
}

In [392]:
'''
for_model = {}
i = 0

for num_page in range(1, 400):
    print('.', end='')
    requestJson = {"section":"used", "category":"cars", "geo_radius":200, "geo_id":[213], "page": num_page }
    response = requests.post(url, json = requestJson, headers = HEADERS)
    data = response.json()
    if data['offers'] == []:
        break
    else:
        for elem in data['offers']:
            def from_elem_to_dict(d_elem):
                d_dict = {}
                d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
                d_dict['model_name'] = d_elem['vehicle_info']['model_info']['code']
                return d_dict
            for_model[i] = from_elem_to_dict(elem)
            i += 1
'''

'\nfor_model = {}\ni = 0\n\nfor num_page in range(1, 400):\n    print(\'.\', end=\'\')\n    requestJson = {"section":"used", "category":"cars", "geo_radius":200, "geo_id":[213], "page": num_page }\n    response = requests.post(url, json = requestJson, headers = HEADERS)\n    data = response.json()\n    if data[\'offers\'] == []:\n        break\n    else:\n        for elem in data[\'offers\']:\n            def from_elem_to_dict(d_elem):\n                d_dict = {}\n                d_dict[\'brand\'] = d_elem[\'vehicle_info\'][\'mark_info\'][\'code\']\n                d_dict[\'model_name\'] = d_elem[\'vehicle_info\'][\'model_info\'][\'code\']\n                return d_dict\n            for_model[i] = from_elem_to_dict(elem)\n            i += 1\n'

In [393]:
#models2 = pd.DataFrame(for_model).T.groupby("brand")["model_name"].unique()
#models2.to_csv('all_car_brands_and_their_models_from_auto_ru.csv')

In [394]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)['model_name']
df_marks_and_models.head

<bound method NDFrame.head of brand
SKODA    ['OCTAVIA']
Name: model_name, dtype: object>

In [395]:
marks = set(df_marks_and_models.keys())
len(marks)

1

Парсинг

In [396]:
auto_ru = {}
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов


# но сначала установим главные параметры запроса
mark = 'SKODA'  # марка
model = 'OCTAVIA'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "1"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

req_body = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "used",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days": "900",
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=req_body, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    ad = {}
    print(ad)
    #auto_ru[0] = ad
    break
    
#df_auto_ru = pd.DataFrame(auto_ru).T
#df_auto_ru.to_csv('all_auto_ru_07_12_2020-5.csv', index=False)

{}


In [401]:
iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.ctime()
for mark in marks:
    start_time = time.ctime()
    ms = df_marks_and_models[mark].replace("'", '"').replace('" "', '", "').replace('"\n "', '", "')
    # print(ms)
    list_of_models = json.loads(ms)
    for model in list_of_models:
        for num_page in range(1, 200):
            request = {
              "catalog_filter": [
                {
                  "mark": mark,
                  "model": model
                }
              ],
              "section": "used",
              "category": "cars",
              "output_type": "table",
              "geo_radius": 200,
              "geo_id": [
                213
              ]
            }
            response = requests.post(url, json=request, headers = HEADERS)
            data = response.json()
            if data['offers'] == []:
                break
            else:
                print('.', end='')
                for elem in data['offers']:
                    # print(elem)
                    def from_elem_to_dict(d_elem, d_dict):
                        try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
                        except: d_dict['bodyType'] = None

                        try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
                        except: d_dict['brand'] = None

                        try: d_dict['color'] = d_elem['color_hex']
                        except: d_dict['color'] = None

                        try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
                        except: d_dict['fuelType'] = None

                        try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
                        except: d_dict['modelDate'] = None

                        try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
                        except: d_dict['name'] = None

                        try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
                        except: d_dict['numberOfDoors'] = None

                        try: d_dict['productionDate'] = d_elem['documents']['year']
                        except: d_dict['productionDate'] = None

                        try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
                        except: d_dict['vehicleTransmission'] = None

                        try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
                        except: d_dict['engineDisplacement'] = None

                        try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
                        except: d_dict['vehicleConfiguration'] = None

                        try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
                        except: d_dict['enginePower'] = None

                        try: d_dict['description'] = d_elem['description']
                        except: d_dict['description'] = None

                        try: d_dict['mileage'] = d_elem['state']['mileage']
                        except: d_dict['mileage'] = None

                        try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
                        except: d_dict['Комплектация'] = None

                        try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
                        except: d_dict['Привод'] = None

                        try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
                        except: d_dict['Руль'] = None

                        try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
                        except: d_dict['Владельцы'] = None

                        try: d_dict['ПТС'] = d_elem['documents']['pts']
                        except: d_dict['ПТС'] = None

                        try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
                        except: d_dict['Таможня'] = None

                        try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
                        except: d_dict['Владение'] = None

                        try: d_dict['price'] = d_elem['price_info']['RUR']
                        except: d_dict['price'] = None

                        try: d_dict['Состояние'] = d_elem['state_not_beaten']
                        except: d_dict['Состояние'] = None
                            
                        # дата размещения объявления решил добавить 
                        try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
                        except: d_dict['start_date'] = None

                        # статус объявления 
                        try: d_dict['hidden'] =  d_elemelem['additional_info']['hidden']
                        except: d_dict['hidden'] = None

                        # модель 
                        d_dict['model_name'] =  model

                        return d_dict
                    '''
                    ad = {
                        'bodyType': elem['vehicle_info']['configuration']['body_type'],
                        'brand': mark, 
                        #'car_url',
                        'color': color(),
                        #'complectation_dict': '',
                        'description': '', 
                        'engineDisplacement': '',
                        'enginePower': '', 
                        #'equipment_dict': '',
                        'fuelType': '', 
                        #'image': '', 
                        'mileage': '', 
                        'modelDate': '',
                        #'model_info': '',
                        'model_name': '',
                        'name': '', 
                        'numberOfDoors': '',
                        #'parsing_unixtime': '',
                        #'priceCurrency': '',
                        'productionDate': '', 
                        #'sell_id': '', 
                        #'super_gen': '',
                        'vehicleConfiguration': '',
                        'vehicleTransmission': '', 
                        #'vendor': '',
                        'Владельцы': '',
                        'Владение': '', 
                        'ПТС': '',
                        'Привод': '', 
                        'Руль': '', 
                        'Состояние': '',
                        'Таможня': ''}
                    '''
                    auto_ru[iter_of_ads] = from_elem_to_dict(elem, {})
                    iter_of_ads += 1
                #time.sleep(3)
print (f'Всего спарсили {iter_of_ads} строк, начало {start_time_global} конец {time.ctime()}')            

.....

KeyboardInterrupt: 

In [ ]:
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('all_auto_ru_07_12_2020-200.csv', index=False)
df_auto_ru.head
#print(f'кол-во строк датасета:= {len(df_auto_ru)}')

In [ ]:
print(len(test_cars))
result = pd.concat([test_cars, df_auto_ru])
print(len(result))